In [ ]:
def binning(filename = None, savelocation = None, diff = None, x_bin_size = None, y_bin_size = None, 
            min_frame = None, max_frame = None):    
    
    import numpy as np, pandas as pd, csv, os
    
    '''
    Create bins and average values within a bin. 
    '''
    
    diff = int(diff)
    
    #read the csv file
   
    df = pd.read_csv(filename)

    #order the dataset as x, y ,rate for x, rate for y
    df = df[['x', 'y', 'Rate_x', 'Rate_y']]
    
    save_name = ''
    
    '''
    if min_frame and max_frame is not None:
        df.drop(df[df['FRAME'] <= min_frame].index, inplace = True)
        df.drop(df[df['FRAME'] >= max_frame].index, inplace = True)
        save_name = save_name + 'min_frame: {}, maxframe: {}'.format(min_frame, max_frame)
    '''
    
    #define the bin edges from minimum to maximum value
    x_bin_edges = list(range(int(min(df.iloc[:,0])), int(max(df.iloc[:,0])), x_bin_size)) 
    y_bin_edges = list(range(int(min(df.iloc[:,1])), int(max(df.iloc[:,1])), y_bin_size)) 

    #bin columns x and y and assign bin labels
    df['x'] = pd.cut(df.iloc[:,0], bins=x_bin_edges, labels=x_bin_edges[:-1]) 
    df['y'] = pd.cut(df.iloc[:,1], bins=y_bin_edges, labels=y_bin_edges[:-1])

    #convert bin columns to numeric 
    df['x'] = pd.to_numeric(df['x'])
    df['y'] = pd.to_numeric(df['y'])
    df['angle'] = np.arctan2(df['Rate_x'], df['Rate_y'])
    df['magnitude'] = np.sqrt(df['Rate_x']**2.+df['Rate_y']**2)
    
    #df['max_lifetime'] = df.groupby(['x', 'y'])['LIFETIME'].max().reset_index(name='LIFETIME')
    #df['spot_start'] = (df_1['y'].diff() > 0).astype(int)
    #df['spot_id'] = df_1['spot_start'].cumsum()

    #group by the bins and calculate the mean of rates for each bin
    df1 = df.groupby(['x', 'y'])['Rate_x'].mean().reset_index()  #sum will bias bins with more count
    df2 = df.groupby(['x', 'y'])['Rate_y'].mean().reset_index()
    
    df3 = df.groupby(['x', 'y'])['Rate_y'].size().reset_index(name='counts_in_bin')
    
    df4 = df.groupby(['x', 'y'])['angle'].std(ddof=0).reset_index(name='std_angle')
    df5 = df.groupby(['x', 'y'])['magnitude'].std(ddof=0).reset_index(name='std_magnitude') #this is measure of deviation of the strength which isn't comparable to magnitude calculated below
    
    #for a comparable magnitude for this standard deviation calculate mean of all magnitude: df.groupby(['x', 'y'])['magnitude'].mean().reset_index(name='magnitude_strength')
    df6 = df.groupby(['x', 'y'])['Rate_x'].std(ddof=0).reset_index(name='std_ratex')
    df7 = df.groupby(['x', 'y'])['Rate_y'].std(ddof=0).reset_index(name='std_ratey')
    
    #make a databse with rate for each component and number of entries in each bin
    df_binned = pd.concat([df1['x'], df1['y'], df1['Rate_x'], df2['Rate_y'], df6['std_ratex'], df7['std_ratey'], df4['std_angle'], df5['std_magnitude'], df3['counts_in_bin']], axis=1)
    df_binned.drop(df_binned[df_binned['counts_in_bin'] <= 1].index, inplace = True)

    #calculate magnitude and angle for the mean of rates in each bin

    df_binned['mean_magnitude'] = np.sqrt(df_binned['Rate_x']**2 + df_binned['Rate_y']**2)
    df_binned['mean_angle'] = np.arctan2(df_binned['Rate_y'], df_binned['Rate_x'])
    
    df_binned['sem_angle'] = df4['std_angle']/df3['counts_in_bin']
    df_binned['sem_magnitude'] = df5['std_magnitude']/df3['counts_in_bin']
    df_binned['sem_ratex'] = df6['std_ratex']/df3['counts_in_bin']
    df_binned['sem_ratey'] = df7['std_ratey']/df3['counts_in_bin']
    
    
    df_binned['snr_angle'] = np.abs(df_binned['mean_angle']/df_binned['sem_angle'])
    df_binned['snr_magnitude'] = np.abs(df_binned['mean_magnitude']/df_binned['sem_magnitude'])
    df_binned['snr_ratex'] = np.abs(df_binned['Rate_x']/df_binned['sem_ratex'])
    df_binned['snr_ratey'] = np.abs(df_binned['Rate_y']/df_binned['sem_ratey'])
    
    ##df_binned.drop(index=df.index[0], axis=0, inplace=True)

    df_binned.to_csv(savelocation)
    
    return 'Binning Completed'
        

In [ ]:
def Phase_portrait(filename = None, savelocation = None, diff = None, threshold = None, 
                   x_axis = 'Actin', y_axis = 'Wasp', 
                   heatmap = 'counts_in_bin'):   
    
    import numpy as np, matplotlib.pyplot as plt, pandas as pd, matplotlib.colors, csv, os
    
    '''Outputs phase potrait with background.'''
    
    diff = int(diff)
    threshold = int(threshold)
    
    df = pd.read_csv(filename)

    #threshold data
    df.drop(df[df['counts_in_bin'] <= threshold].index, inplace = True)

    #initiate plot design
    fig, axs = plt.subplots(1, 1, figsize=(13, 26), dpi = 400)

    #heatmap with colorbar
    df['heatmap'] = df[heatmap]
    counts_values = df.pivot_table(values='heatmap', index='y', columns='x')
    heatmap = axs.pcolormesh(counts_values.columns, counts_values.index, counts_values.values, cmap='rainbow', alpha=0.5) 
                             #vmin=-0.001, vmax=0.001)
    heatbar = plt.colorbar(heatmap, orientation='horizontal', ax=axs, pad=0.05 )
    heatbar.set_label('Heatmap of {}'.format(heatmap), fontsize=20)

    #create a colormap based on the magnitude for plotting vecotr
    magnitude_norm = matplotlib.colors.Normalize(vmin=df['mean_magnitude'].min(), vmax=df['mean_magnitude'].max())
    cmap = plt.cm.seismic 
    sm = plt.colorbar(plt.cm.ScalarMappable(cmap=cmap, norm=magnitude_norm),
                      ax=axs, orientation='vertical', pad=0.05) 
    sm.set_ticks(ticks = [df['mean_magnitude'].min(), df['mean_magnitude'].mean(),  df['mean_magnitude'].max()], 
                 labels = ['Minimum Magnitude', 'Mean Magnitude', 'Maximum Magnitude'], fontsize = 12,
                minor=True)
    sm.set_label('Magnitude', fontsize = 20)
    
    #sm.set_array([])  #required for ScalarMappable don't know why
    #fig.colorbar(sm, label='Magnitude')

    #plotting vectors
    df['dx'] = np.cos(df['mean_angle'])
    df['dy'] = np.sin(df['mean_angle'])

    plt.quiver(df['x'], df['y'], df['dx'], df['dy'], df['mean_magnitude'], 
               angles='xy', width = 0.02, headwidth = 20, headlength = 50, headaxislength = 40,  
               cmap=cmap, norm=magnitude_norm)

    
    #setting plot details
    axs.set_xlabel(x_axis, fontsize = 20, fontstyle= 'normal')
    axs.set_ylabel(y_axis, fontsize = 20, fontstyle= 'normal')
    axs.tick_params(axis='both', which='major', labelsize=15, direction='out', length=10, width=2)
    axs.set_title('{}  \n with threshold {}, \n heatmap of {}'.format(filename, threshold, heatmap), 
                  pad = 20, fontsize = 22, weight = 'heavy')
    
    #x = np.array(range(24))
    #axs.plot(x,2*x,linewidth=7,color='b')
    #axs.plot(x,3*x,linewidth=7,color='g')
    
    #axs.grid()
    #axs.set_xlim(-1,25)
    #axs.set_ylim(-1,60)
    
    fig.savefig(savelocation, format = 'png', dpi = 400)    
    return "Potrait saved as a png"

